[View in Colaboratory](https://colab.research.google.com/github/Hoiy/kaggle-colab-env/blob/master/template.ipynb)

In [0]:
import pandas as pd

In [0]:
train = pd.read_csv('./data/train.csv.zip')
test = pd.read_csv('./data/test.csv.zip')
submission = pd.read_csv('./data/sample_submission.csv')

In [9]:
train.shape, test.shape

((1503424, 18), (508438, 17))

In [10]:
train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [66]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, Conv1D, Reshape, MaxPooling1D, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.constraints import non_neg, unit_norm
import keras.backend as K
from keras.metrics import mse
import tensorflow as tf

def rmse(y_true, y_pred):
    # bug when K.sqrt(mse(y_true, y_pred))
    return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_true, y_pred))))

def build_model():
    inp = Input(shape=(1, ))
    hidden = Dense(1, activation='relu')(inp)
    out = Dense(1, activation="sigmoid")(hidden)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(loss=rmse,
                  optimizer='adam')
    return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
dense_82 (Dense)             (None, 1)                 2         
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 2         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [67]:
BATCH_SIZE = 4096
EPOCHS = 20000
FILE_PATH="best.hdf5"

checkpoint = ModelCheckpoint(FILE_PATH, monitor='val_loss', save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early] #early

model = build_model()
model.fit(
    train['image_top_1'], 
    train['deal_probability'], 
    validation_split=0.2,
    shuffle=True, 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS, 
    callbacks=callbacks_list)

Train on 1202739 samples, validate on 300685 samples
Epoch 1/20000
1202739/1202739 [==============================] - 3s 2us/step - loss: 0.4163 - val_loss: 0.3894
Epoch 2/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.3657 - val_loss: 0.3447
Epoch 3/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.3274 - val_loss: 0.3127
Epoch 4/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.3010 - val_loss: 0.2917
Epoch 5/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2843 - val_loss: 0.2787
Epoch 6/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2740 - val_loss: 0.2710
Epoch 7/20000
 745472/1202739 [=================>............] - ETA: 0s - loss: 0.2690

1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2680 - val_loss: 0.2664
Epoch 8/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2644 - val_loss: 0.2637
Epoch 9/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2624 - val_loss: 0.2622
Epoch 10/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2612 - val_loss: 0.2614
Epoch 11/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2606 - val_loss: 0.2609
Epoch 12/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2602 - val_loss: 0.2607
Epoch 13/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2601 - val_loss: 0.2605
Epoch 14/20000
 307200/1202739 [======>.......................] - ETA: 1s - loss: 0.2603

1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2600 - val_loss: 0.2605
Epoch 15/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2600 - val_loss: 0.2605
Epoch 16/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 17/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 18/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 19/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 20/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 21/20000
 188416/1202739 [===>..........................] - ETA: 1s - loss: 0.2612

1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 22/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 23/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 24/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 25/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 26/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605
Epoch 27/20000
1202739/1202739 [==============================] - 2s 2us/step - loss: 0.2599 - val_loss: 0.2605


In [68]:
from keras.models import load_model
model = load_model(FILE_PATH, custom_objects={'rmse': rmse})

model.evaluate(
    train['image_top_1'], 
    train['deal_probability'],
    batch_size=BATCH_SIZE
)

1503424/1503424 [==============================] - 1s 1us/step


0.2600449546134455

In [0]:
SUBMISSION_FILE='baseline.csv'
SUBMISSION_MESSAGE='"Baseline 0.2600"'

submission['deal_probability'] = model.predict(
    pd.merge(submission, test[['item_id', 'image_top_1']], how='left', on='item_id')['image_top_1'],
    batch_size=BATCH_SIZE
)
submission.to_csv(SUBMISSION_FILE, index=False)

In [89]:
!kaggle competitions submit -f {SUBMISSION_FILE} -m {SUBMISSION_MESSAGE}

Using competition: avito-demand-prediction
Successfully submitted to Avito Demand Prediction Challenge